In [3]:
import pandas as pd
import numpy as np
import sys, os

from matplotlib import pyplot as plt
sys.path.insert(1, os.path.join(sys.path[0], '../../'))  # for importing local packages from src
sys.path.insert(1, os.path.join(sys.path[0], '../'))  # for importing model config

%reload_ext autoreload
%autoreload 2 

## Datasets

In [4]:
from src.model.rc_dataset import ExperimentDataset


DATASET_NEMA_H0 = '../datasets/dataset_NEMA_NEMA_H0.csv'
DATASET_NEMA_H3 = '../datasets/dataset_NEMA_NEMA_H3.csv'
DATASET_NEMA_H15 = '../datasets/dataset_NEMA_NEMA_H15.csv'

dataset_nema_h0 = ExperimentDataset(csv_path=DATASET_NEMA_H0)
dataset_nema_h3 = ExperimentDataset(csv_path=DATASET_NEMA_H3)
dataset_nema_h15 = ExperimentDataset(csv_path=DATASET_NEMA_H15)

datasets = [
  dataset_nema_h0, 
  dataset_nema_h3,
  dataset_nema_h15
]

## Model configuration

In [5]:
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import LeaveOneGroupOut
from src.learning.scorers import nmse_scorer
from pipeline_cnwheat import GroupGenerator, TimeGenerator
from pipeline_base import TrainTestSplitter, DirectTransform, WarmupTransform

# constants
DATASET_ID = 'NEMA_combined'
STATE_SIZE = 7
N_STATE_SAMPLES = 16
state_ids = None

# Reservoir generation
np.random.seed(42)
# NOTE: all measurable reservoirs considered have the same reservoir size of 10.
generate_state_sample = lambda : np.random.choice(10, size=STATE_SIZE, replace=False)


# Readout model
readout_model = Pipeline([
  ('ridge_regression', Ridge(alpha=1, fit_intercept=True))
])
model_param_grid = [{
  'ridge_regression__alpha': 10 ** np.linspace(np.log10(1e-4), np.log10(1e2), 50)
}]

# Regression task pipeline
shared_pipeline_params = {
  # Data generation
  'datasets': datasets,
  'groups': GroupGenerator(day_length=24),
  'time': TimeGenerator(day_length=24),
  
  # Model training and validation
  'readout_model': readout_model,
  'model_param_grid': model_param_grid,
  'model_scorer': nmse_scorer,
  'folds': LeaveOneGroupOut(),
  'train_test_split': TrainTestSplitter(block_size=4, test_ratio=0.5),
}

## Regression Pipelines

In [6]:
from pipeline_base import (
  RCPipeline, 
  Rescale, 
  DaylightMask
)

from pipeline_cnwheat import (
  TargetGenerator, 
  SingleReservoirGenerator,
  MultiReservoirGenerator,
  TargetReservoirGenerator,
  GroupRescale
)

from model_config_cnwheat import (
  baseline_reservoirs,
  heterogeneous_reservoirs, 
  final_targets, 
  measurable_reservoirs,
)

### Computational Benchmarks

In [7]:
from pipeline_base import DelayLineTransform, CustomWarmupTransform
from pipeline_base import PolynomialTargetTransform
from pipeline_base import NarmaTargetTransform


def delay_line_benchmark(target_gen, target_id, res_gen, res_id, rescale, *, delay_steps: [int]):
  for d in delay_steps:
    pipeline = RCPipeline(
      metadata={
        'target_id': target_id, 
        'reservoir_id': res_id, 
        'dataset_id': DATASET_ID,
        'benchmark': 'delay_line',
        'delay': d,
      },
      # Data generation
      target=target_gen,
      reservoir=res_gen,
      # Data transformation
      transforms=[
        # NOTE: remove warmup before transform, otherwise the 
        # warmup steps are used as delayed prediction targets.
        WarmupTransform(warmup_days=4, day_length=24),
        DelayLineTransform(delay_steps=d),
      ],
      # Data preprocessing
      preprocessing=[
        DaylightMask(day_length=24, start=5, end=21),
        rescale
      ],
      **shared_pipeline_params,
    )
    yield pipeline


def polynomial_benchmark(target_gen, target_id, res_gen, res_id, rescale, *, exponents: [int]):
  for e in exponents:
    pipeline = RCPipeline(
      metadata={
        'target_id': target_id, 
        'reservoir_id': res_id, 
        'dataset_id': DATASET_ID,
        'benchmark': 'polynomial',
        'exponent': e,
      },
      # Data generation
      target=target_gen,
      reservoir=res_gen,
      # Data transformation
      transforms=[
        WarmupTransform(warmup_days=4, day_length=24),
        PolynomialTargetTransform(poly_coefs=[0] * e + [1]),
      ],
      # Data preprocessing
      preprocessing=[
        DaylightMask(day_length=24, start=5, end=21),
        rescale
      ],
      **shared_pipeline_params,
    )
    yield pipeline


def narma_benchmark(target_gen, target_id, res_gen, res_id, rescale, *, n_values: [int]):
  for n in n_values:
    pipeline = RCPipeline(
      metadata={
        'target_id': target_id, 
        'reservoir_id': res_id, 
        'dataset_id': DATASET_ID,
        'benchmark': 'NARMA',
        'narma_n': n,
      },
      # Data generation
      target=target_gen,
      reservoir=res_gen,
      # Data transformation
      transforms=[
        # NOTE: remove warmup before transform, otherwise the 
        # warmup steps are used as input to the NARMA system.
        WarmupTransform(warmup_days=4, day_length=24),  
        NarmaTargetTransform(n=n, scale=1),
      ],
      # Data preprocessing
      preprocessing=[
        DaylightMask(day_length=24, start=5, end=21),
        rescale
      ],
      **shared_pipeline_params,
    )
    yield pipeline

In [8]:
benchmarks = [
  {
    'benchmark': delay_line_benchmark,
    'kwargs': {
      'delay_steps': np.arange(0, 9)  # 0-8 hours
    }
  },
  {
    'benchmark': polynomial_benchmark,
    'kwargs': {
      'exponents': np.arange(1, 10)  # exponents 1-9
    }
  },
  {
    'benchmark': narma_benchmark,
    'kwargs': {
      'n_values': [2, 4, 8, 12, 18, 24]
      # 'n_values': [2, 4, 6, 8, 10, 12, 18, 20, 24]
    }
  },
]

In [9]:
def generate_benchmark_pipelines(target_gen, target_id, reservoir_gen, reservoir_id, rescale):
  pipeline_generators = []
  for benchmark in benchmarks:
    gen = benchmark['benchmark'](target_gen, target_id, reservoir_gen, reservoir_id, rescale, **benchmark['kwargs'])
    pipeline_generators.append(gen)
  return chain(*pipeline_generators)

In [14]:
from model_config_cnwheat import input_targets

target_state_pairs = [(target, state_var) for target in input_targets for state_var in measurable_reservoirs]
target_env_pairs = [(name, target, env_targets) for target in input_targets for (name, env_targets) in baseline_reservoirs]
target_het_pairs = [(name, target, state_vars) for target in input_targets for (name, state_vars) in heterogeneous_reservoirs]

print(input_targets)
print(measurable_reservoirs)
print(baseline_reservoirs)
print(heterogeneous_reservoirs)

['input_air_temperature', 'input_humidity', 'input_PARi']
['state__An', 'state__Ts', 'state__gs', 'state__Tr']
[('env_all', ['input_air_temperature', 'input_humidity', 'input_PARi']), ('env_temp', ['input_air_temperature']), ('env_humidity', ['input_humidity']), ('env_PAR', ['input_PARi'])]
[('state__het_all', ('state__An', 'state__Transpiration', 'state__Ts', 'state__gs', 'state__Ag', 'state__Tr', 'state__Rd', 'state__sum_respi', 'state__PARa'))]


In [13]:
from itertools import chain


def generate_pipelines(state_ids):
  all_pipelines = []

  for target, state_var in target_state_pairs:
    target_gen = TargetGenerator(target=target)
    res_gen = SingleReservoirGenerator(state_var=state_var, state_ids=state_ids)
    pipelines = generate_benchmark_pipelines(
      target_gen, target, res_gen, state_var, 
      Rescale(per_feature=False)
    )
    all_pipelines += pipelines

  for name, target, state_vars in target_het_pairs:
    target_gen = TargetGenerator(target=target)
    res_gen = MultiReservoirGenerator(state_vars=state_vars, state_ids=state_ids)
    pipelines = generate_benchmark_pipelines(
      target_gen, target, res_gen, name, 
      GroupRescale(datasets=datasets, state_vars=state_vars)
    )
    all_pipelines += pipelines

  for name, target, env_targets in target_env_pairs:
    target_gen = TargetGenerator(target=target)
    res_gen = TargetReservoirGenerator(targets=env_targets)
    pipelines = generate_benchmark_pipelines(
      target_gen, target, res_gen, name, 
      Rescale(per_feature=True)
    )
    all_pipelines += pipelines
  
  return all_pipelines


np.random.seed(42)
state_ids = generate_state_sample()
pipelines = generate_pipelines(state_ids)
print(f'Total pipelines: {len(pipelines)}')
print(f'Total runs: {len(pipelines) * N_STATE_SAMPLES}')

Total pipelines: 648
Total runs: 10368


## Execution

In [15]:
pipes = generate_pipelines(state_ids)
n_pipelines = len(pipes)
print(n_pipelines)

648


In [16]:
from tqdm import tqdm
from itertools import chain
from pipeline_base import execute_pipeline


RESULTS_FILE = 'results_cnwheat_comp_bench_16sample.csv'

np.random.seed(42)
all_state_samples = [generate_state_sample() for _ in range(N_STATE_SAMPLES)]
total_loops = n_pipelines * N_STATE_SAMPLES
results = []


with tqdm(total=total_loops) as pbar:
  for i_state_sample, state_ids in enumerate(all_state_samples):
    pipelines = generate_pipelines(state_ids)
    for pipeline in pipelines:
      result = execute_pipeline(pipeline)
      result['state_sample'] = i_state_sample
      results.append(result)
      pbar.update(1)


results_df = pd.DataFrame.from_dict(results)
results_df.to_csv(RESULTS_FILE)
print(f'Saved scores to {RESULTS_FILE}')

  3%|▎         | 288/10368 [02:06<1:21:50,  2.05it/s]f:\thesis-cyborg-plants\notebooks\regression_tests\..\pipeline_cnwheat.py:217: RuntimeWarning: Mean of empty slice.
  X_g = (X_g - X_g.mean()) / X_g.std()
C:\Users\Max\anaconda3\envs\rc-plants\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Max\anaconda3\envs\rc-plants\lib\site-packages\numpy\core\_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Max\anaconda3\envs\rc-plants\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\Max\anaconda3\envs\rc-plants\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  3%|▎         | 289/10

Saved scores to results_cnwheat_comp_bench_16sample.csv
